In [1]:
import folium
import pandas as pd
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [4]:
file_path = r"C:\Users\moroi\OneDrive\Desktop\dataset_part_2.csv"
df = pd.read_csv(file_path)

In [5]:
# Get unique launch sites with their coordinates
launch_sites = df[['LaunchSite', 'Latitude', 'Longitude']].drop_duplicates().reset_index(drop=True)

# Center the map around the average coordinates of all sites
map_center = [launch_sites['Latitude'].mean(), launch_sites['Longitude'].mean()]
site_map = folium.Map(location=map_center, zoom_start=4)

# Add markers for each launch site
for index, row in launch_sites.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['LaunchSite'],
        icon=folium.Icon(color='blue', icon='rocket', prefix='fa')
    ).add_to(site_map)

site_map


In [6]:
# Create a fresh map centered around the average location
launch_map = folium.Map(location=map_center, zoom_start=4)

# Group by launches
for _, row in df.iterrows():
    location = [row['Latitude'], row['Longitude']]
    launch_result = "Success" if row['Class'] == 1 else "Failure"
    color = 'green' if row['Class'] == 1 else 'red'
    
    folium.CircleMarker(
        location=location,
        radius=5,
        popup=f"{row['LaunchSite']} - {launch_result}",
        color=color,
        fill=True,
        fill_opacity=0.7
    ).add_to(launch_map)

launch_map


In [9]:
import pandas as pd
import dash
from dash import dcc, html, Input, Output
import plotly.express as px

# Load dataset

df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year

# Initialize Dash app
app = dash.Dash(__name__)

# Create dropdown options
site_options = [{'label': site, 'value': site} for site in df['LaunchSite'].unique()]
site_options.insert(0, {'label': 'All Sites', 'value': 'ALL'})

# Layout
app.layout = html.Div([
    html.H1("SpaceX Launch Records Dashboard", style={'textAlign': 'center'}),

    # Dropdown
    dcc.Dropdown(id='site-dropdown',
                 options=site_options,
                 value='ALL',
                 placeholder="Select a Launch Site",
                 searchable=True),

    # Pie chart
    dcc.Graph(id='success-pie-chart'),

    # Payload range slider
    html.P("Payload range (Kg):"),
    dcc.RangeSlider(id='payload-slider',
                    min=0, max=10000, step=500,
                    marks={0: '0', 2500: '2500', 5000: '5000', 7500: '7500', 10000: '10000'},
                    value=[0, 10000]),

    # Scatter plot
    dcc.Graph(id='success-payload-scatter')
])

# Callback for pie chart
@app.callback(
    Output('success-pie-chart', 'figure'),
    Input('site-dropdown', 'value')
)
def update_pie_chart(site):
    if site == 'ALL':
        fig = px.pie(df, names='LaunchSite', values='Class',
                     title='Total Success Launches by Site')
    else:
        filtered_df = df[df['LaunchSite'] == site]
        fig = px.pie(filtered_df, names='Class',
                     title=f'Total Launch Outcomes for {site}')
    return fig

# Callback for scatter plot
@app.callback(
    Output('success-payload-scatter', 'figure'),
    [Input('site-dropdown', 'value'),
     Input('payload-slider', 'value')]
)
def update_scatter_plot(site, payload_range):
    low, high = payload_range
    filtered_df = df[(df['PayloadMass'] >= low) & (df['PayloadMass'] <= high)]
    if site != 'ALL':
        filtered_df = filtered_df[filtered_df['LaunchSite'] == site]
    fig = px.scatter(filtered_df, x='PayloadMass', y='Class', color='BoosterVersion',
                     title='Payload vs. Outcome by Booster Version')
    return fig


if __name__ == '__main__':
 app.run_server(debug=True)
